# [Bonus Tasks](svm_segm.ipynb)

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import scipy.ndimage as ndi
import skimage.morphology as morph
import skimage.util
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [2]:
def blend_result(img, seg, border_radius=1, interior_opacity=1, interior_opacity_decay=0.9, color=(0,1,0)):
    img  = np.dstack([img] * 3).copy()
    img -= img.min()
    img /= img.max()
    selem  = morph.disk(border_radius)
    seg_bd = np.logical_xor(morph.binary_dilation(seg, selem), morph.binary_erosion(seg, selem))
    mask_decay = ndi.distance_transform_edt(seg)
    for i in range(3):
        opacity = interior_opacity / pow(1 + mask_decay[seg], interior_opacity_decay)
        img[:,:,i][seg] = color[i] * opacity + (1 - opacity) * img[:,:,i][seg]
        img[:,:,i][seg_bd] = color[i]
    return img

In [3]:
patch_sizes = (32, 32)

---

**[Task 1.1.]()** Implement `create_data_matrix`:

In [4]:
# Implement the function create_data_matrix, which takes an image as the pa-
# rameter, creates the corresponding data matrix 𝑋 for that image, and returns the
# data matrix – Hint: Loops can be avoided by using the function view_as_blocks6
# from the skimage.util module.
def create_data_matrix(image):
    patches = skimage.util.view_as_blocks(image, block_shape=patch_sizes)
    n_blocks_x, n_blocks_y, _, _ = patches.shape
    data_matrix = patches.reshape(n_blocks_x * n_blocks_y, -1)
    return data_matrix

**[Task 1.2.]()** Implement `create_gt_labels_vector`:

In [5]:
# Implement the function create_gt_labels_vector, which takes a binary image
# as the parameter (the ground truth segmentation), creates the vector of labels 𝑦,
# and returns that vector. The implementation is analogous to create_data_matrix:
# If the area of the ground truth image corresponding to an image patch contains
# more than 50% foreground, assign the label +1 to that image patch. If that area
# contains no foreground, assign the label −1. Otherwise, assign the label 0 (see
# below).

def create_gt_labels_vector(gt_image):
    patches = skimage.util.view_as_blocks(gt_image, block_shape=patch_sizes)
    n_blocks_x, n_blocks_y, _, _ = patches.shape
    labels_vector = np.zeros(n_blocks_x * n_blocks_y, dtype=int)
    for i in range(n_blocks_x):
        for j in range(n_blocks_y):
            patch = patches[i, j]
            foreground_ratio = np.sum(patch) / patch.size
            if foreground_ratio > 0.5:
                labels_vector[i * n_blocks_y + j] = 1
            elif foreground_ratio == 0:
                labels_vector[i * n_blocks_y + j] = -1
            else:
                labels_vector[i * n_blocks_y + j] = 0
    return labels_vector

**[Task 1.3.]()** Create the SVM classifier:

In [6]:
clf = make_pipeline(StandardScaler(), SVC(class_weight='balanced', gamma=0.1))

**[Task 1.3 (a).]()** Create the data matrices for the images `dna-33` and `dna-44`:

In [7]:
# Using the function create_data_matrix implemented before, generate the
# data matrices for the two images data/NIH3T3/im/dna-33.png and dna-44.png.

dna_33 = plt.imread('data/NIH3T3/im/dna-33.png')
dna_44 = plt.imread('data/NIH3T3/im/dna-44.png')

matrix_33 = create_data_matrix(dna_33)
matrix_44 = create_data_matrix(dna_44)

**[Task 1.3 (b).]()** Create the corresponding ground truth label vectors:

In [8]:
# Use create_gt_labels_vector to generate the label vectors based on the
# corresponding ground truth images data/NIH3T3/gt/33.png and 44.png.

gt_33 = plt.imread('data/NIH3T3/gt/33.png')
gt_44 = plt.imread('data/NIH3T3/gt/44.png')

labels_33 = create_gt_labels_vector(gt_33)
labels_44 = create_gt_labels_vector(gt_44)

**[Task 1.3 (c).]()** Create the *combined* data matrices and ground truth label vectors:

In [9]:
# Stack the two data matrices and the two corresponding label vectors, but
# only keeping those rows of the data matrices and the label vectors, where
# the corresponding label is −1 or +1 (i.e. discard rows where the label is 0) –
# Hint: You can use np.concatenate7 to perform the stacking.

X_train = np.concatenate((matrix_33[labels_33 != 0], matrix_44[labels_44 != 0]), axis=0)
y_train = np.concatenate((labels_33[labels_33 != 0], labels_44[labels_44 != 0]), axis=0)


**[Task 1.3 (d).]()** Train the classifier using the data matrix and label vectors from above:

In [10]:
clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(class_weight='balanced', gamma=0.1))])

**[Task 1.4.]()** Implement the function `predict_image`:

In [11]:
def predict_image(img) :
    # get the matrix from the image
    X = create_data_matrix(img)
    # predict the labels from the patches (is a vector)
    prediction = clf.predict(X)

    # result image with False at every point
    result = np.zeros(img.shape, dtype=bool)
    
    # view from the result in windows (if changed here, also changed in results)
    patch_height, patch_width = patch_sizes
    step = patch_width // 2
    patches = skimage.util.view_as_windows(result, window_shape = patch_sizes, step = step)

    # the patch with all True values
    t = np.full(patch_sizes, True, dtype=bool)

    # go trough all patches
    for i in range(patches.shape[0]) :
        for n in range(patches.shape[1]) :
            # convert the patch index into the corresponding labels index in the vector
            num = i * patches.shape[1] + n
            # if the label in the prediction is 1, change the patch to all True
            if prediction[num] == 1 :       
                patches[i, n] = t
    

    return result


Test your implementation:

In [12]:
img = plt.imread(f'data/NIH3T3/im/dna-0.png')
seg = predict_image(img)

print(seg)

plt.imshow(seg, vmin=0, vmax=1, cmap='gray')

plt.figure()
plt.imshow(blend_result(img, seg))

IndexError: index 1344 is out of bounds for axis 0 with size 1344

**[Task 1.5.]()** Perform batch processing:

In [ ]:
seq = [28, 29, 33, 44, 46, 49]

all = []

for i in seq :
    img = plt.imread(f'data/NIH3T3/im/dna-{i}.png')
    ground = plt.imread(f'data/NIH3T3/gt/{i}.png')

    binground = ground > 0.5
    binimg = predict_image(img)
    
    dice = 2 * (binimg * binground).sum() / (binimg.sum() + binground.sum())

    all.append(dice)

    print('image: ' + str(i) + ', Dice: ' + str(round(dice, 2)))

print()
print('Average Dice: ' + str(round(sum(all)/len(all), 2)))
